<a href="https://colab.research.google.com/github/BelovedTemitope/Machine-Learning-Projects/blob/master/Hotspots_Dataset_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('/content/drive/My Drive/UmojaHack#3:Hotspots/train.csv',parse_dates=['date'])
data.head()

,ID,area,date,lat,lon,burn_area,climate_aet,climate_def,climate_pdsi,climate_pet,climate_pr,climate_ro,climate_soil,climate_srad,climate_swe,climate_tmmn,climate_tmmx,climate_vap,climate_vpd,climate_vs,elevation,landcover_0,landcover_1,landcover_2,landcover_3,landcover_4,landcover_5,landcover_6,landcover_7,landcover_8,population_density,precipitation
0,0_2000-04-01,0,2000-04-01,25.447,5.296,0.003688,1250.622712,0.0,-178.916305,1250.622712,158.387765,8.000000,498.913776,2143.430034,0.0,206.935453,323.873675,2508.795159,115.989282,129.825024,590.564102,0.0,0.0,0.350169,0.0,0.649524,0.0,0.000307,0.0,0.0,2.214262,0.198996
1,1_2000-04-01,1,2000-04-01,25.669,5.293,0.000000,1238.019166,0.0,-150.779947,1238.019166,164.645065,8.104576,590.346191,2132.253953,0.0,204.835170,321.838884,2500.432319,112.619789,129.471370,611.544277,0.0,0.0,0.429049,0.0,0.570644,0.0,0.000307,0.0,0.0,3.833042,0.188071
2,2_2000-04-01,2,2000-04-01,25.443,5.074,0.000000,1240.449964,0.0,-200.503858,1240.449964,163.699542,8.027851,536.613938,2139.632626,0.0,207.001206,321.846757,2519.259344,112.378707,127.550639,594.188987,0.0,0.0,0.238300,0.0,0.761700,0.0,0.000000,0.0,0.0,1.927303,0.211730
3,3_2000-04-01,3,2000-04-01,25.665,5.070,0.000000,1229.240077,0.0,-177.011032,1229.240077,172.996043,8.820002,648.271615,2125.206979,0.0,204.127473,319.348123,2493.392613,109.132390,131.703082,645.362481,0.0,0.0,0.488146,0.0,0.511854,0.0,0.000000,0.0,0.0,1.878281,0.215403
4,4_2000-04-01,4,2000-04-01,25.886,5.067,0.000307,1224.093679,0.0,-153.256111,1224.093679,178.396749,9.000000,719.423119,2114.709813,0.0,203.295123,318.639735,2487.984467,108.161469,133.354846,660.372381,0.0,0.0,0.322243,0.0,0.677757,0.0,0.000000,0.0,0.0,1.968818,0.199975


In [ ]:
test = pd.read_csv('/content/drive/My Drive/UmojaHack#3:Hotspots/test.csv', parse_dates=['date'])
test.head(1)

,ID,area,date,lat,lon,burn_area,climate_aet,climate_def,climate_pdsi,climate_pet,climate_pr,climate_ro,climate_soil,climate_srad,climate_swe,climate_tmmn,climate_tmmx,climate_vap,climate_vpd,climate_vs,elevation,landcover_0,landcover_1,landcover_2,landcover_3,landcover_4,landcover_5,landcover_6,landcover_7,landcover_8,population_density,precipitation
0,0_2014-01-01,0,2014-01-01,25.447,5.296,0,142.756021,1217.92618,-953.401012,1360.571291,1.0,0.0,355.59369,2349.499157,0.0,187.541787,332.578998,2542.124518,111.013969,156.563945,590.564102,0.0,0.0,0.430648,0.0,0.569045,0.0,0.000307,0.0,0.0,2.773423,0.000457


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626644 entries, 0 to 626643
Data columns (total 32 columns):
ID                    626644 non-null object
area                  626644 non-null int64
date                  626644 non-null datetime64[ns]
lat                   626644 non-null float64
lon                   626644 non-null float64
burn_area             626644 non-null float64
climate_aet           626644 non-null float64
climate_def           626644 non-null float64
climate_pdsi          626644 non-null float64
climate_pet           626644 non-null float64
climate_pr            626644 non-null float64
climate_ro            626644 non-null float64
climate_soil          626644 non-null float64
climate_srad          626644 non-null float64
climate_swe           626644 non-null float64
climate_tmmn          626644 non-null float64
climate_tmmx          626644 non-null float64
climate_vap           626644 non-null float64
climate_vpd           626644 non-null float64
climate_vs 

In [ ]:
test.burn_area.unique()

array([0])

In [ ]:
data['population_density'].fillna(data['population_density'].mean(),inplace =True)
data['population_density'].isnull().sum()

0

In [ ]:
data.drop(['ID'],axis=1,inplace=True)

In [ ]:
data.head()
data['month'] = data.date.dt.month
#data['year'] = data.date.dt.year

In [ ]:
train_all = data.copy().dropna()
train = train_all.loc[train_all.date < '2011-01-01']
valid = train_all.loc[train_all.date > '2011-01-01']
print(train.shape, valid.shape)

(489088, 32) (133735, 32)


In [ ]:
in_cols = train.columns.drop(['burn_area','date','area'])
target_col = 'burn_area'
in_cols

Index(['lat', 'lon', 'climate_aet', 'climate_def', 'climate_pdsi',
       'climate_pet', 'climate_pr', 'climate_ro', 'climate_soil',
       'climate_srad', 'climate_swe', 'climate_tmmn', 'climate_tmmx',
       'climate_vap', 'climate_vpd', 'climate_vs', 'elevation', 'landcover_0',
       'landcover_1', 'landcover_2', 'landcover_3', 'landcover_4',
       'landcover_5', 'landcover_6', 'landcover_7', 'landcover_8',
       'population_density', 'precipitation', 'month'],
      dtype='object')

In [ ]:
X_train, y_train = train[in_cols], train[target_col]
X_valid, y_valid = valid[in_cols], valid[target_col]
X_train.head()

,lat,lon,climate_aet,climate_def,climate_pdsi,climate_pet,climate_pr,climate_ro,climate_soil,climate_srad,climate_swe,climate_tmmn,climate_tmmx,climate_vap,climate_vpd,climate_vs,elevation,landcover_0,landcover_1,landcover_2,landcover_3,landcover_4,landcover_5,landcover_6,landcover_7,landcover_8,population_density,precipitation,month
0,25.447,5.296,1250.622712,0.0,-178.916305,1250.622712,158.387765,8.000000,498.913776,2143.430034,0.0,206.935453,323.873675,2508.795159,115.989282,129.825024,590.564102,0.0,0.0,0.350169,0.0,0.649524,0.0,0.000307,0.0,0.0,2.214262,0.198996,4
1,25.669,5.293,1238.019166,0.0,-150.779947,1238.019166,164.645065,8.104576,590.346191,2132.253953,0.0,204.835170,321.838884,2500.432319,112.619789,129.471370,611.544277,0.0,0.0,0.429049,0.0,0.570644,0.0,0.000307,0.0,0.0,3.833042,0.188071,4
2,25.443,5.074,1240.449964,0.0,-200.503858,1240.449964,163.699542,8.027851,536.613938,2139.632626,0.0,207.001206,321.846757,2519.259344,112.378707,127.550639,594.188987,0.0,0.0,0.238300,0.0,0.761700,0.0,0.000000,0.0,0.0,1.927303,0.211730,4
3,25.665,5.070,1229.240077,0.0,-177.011032,1229.240077,172.996043,8.820002,648.271615,2125.206979,0.0,204.127473,319.348123,2493.392613,109.132390,131.703082,645.362481,0.0,0.0,0.488146,0.0,0.511854,0.0,0.000000,0.0,0.0,1.878281,0.215403,4
4,25.886,5.067,1224.093679,0.0,-153.256111,1224.093679,178.396749,9.000000,719.423119,2114.709813,0.0,203.295123,318.639735,2487.984467,108.161469,133.354846,660.372381,0.0,0.0,0.322243,0.0,0.677757,0.0,0.000000,0.0,0.0,1.968818,0.199975,4


In [ ]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [ ]:
model = RandomForestRegressor()

In [ ]:
model.fit(X_train, y_train)

# Make predictions
preds = model.predict(X_valid)

# Score
mean_squared_error(y_valid, preds)**0.5 

0.028447118907351333

In [ ]:
test['month'] = test.date.dt.month
#test['year'] = test.date.dt.year
test.head()

,ID,area,date,lat,lon,burn_area,climate_aet,climate_def,climate_pdsi,climate_pet,climate_pr,climate_ro,climate_soil,climate_srad,climate_swe,climate_tmmn,climate_tmmx,climate_vap,climate_vpd,climate_vs,elevation,landcover_0,landcover_1,landcover_2,landcover_3,landcover_4,landcover_5,landcover_6,landcover_7,landcover_8,population_density,precipitation,month
0,0_2014-01-01,0,2014-01-01,25.447,5.296,0,142.756021,1217.926180,-953.401012,1360.571291,1.000000,0.0,355.593690,2349.499157,0.0,187.541787,332.578998,2542.124518,111.013969,156.563945,590.564102,0.0,0.0,0.430648,0.0,0.569045,0.0,0.000307,0.0,0.0,2.773423,0.000457,1
1,1_2014-01-01,1,2014-01-01,25.669,5.293,0,154.836847,1193.866315,-961.703043,1348.688788,1.000000,0.0,382.206636,2340.597389,0.0,185.895903,330.826066,2551.085649,106.576545,158.165429,611.544277,0.0,0.0,0.494624,0.0,0.505068,0.0,0.000307,0.0,0.0,4.697883,0.000129,1
2,2_2014-01-01,2,2014-01-01,25.443,5.074,0,127.790933,1223.640222,-962.435857,1351.521823,1.000000,0.0,318.078852,2351.867856,0.0,188.441645,330.077285,2551.183265,107.104292,153.257656,594.188987,0.0,0.0,0.288343,0.0,0.711657,0.0,0.000000,0.0,0.0,2.488614,0.000398,1
3,3_2014-01-01,3,2014-01-01,25.665,5.070,0,136.924571,1201.258784,-961.326298,1338.120278,1.000000,0.0,339.386497,2348.509414,0.0,185.393692,326.942319,2539.815326,101.962346,155.518048,645.362481,0.0,0.0,0.492103,0.0,0.507897,0.0,0.000000,0.0,0.0,2.439959,0.000158,1
4,4_2014-01-01,4,2014-01-01,25.886,5.067,0,144.278146,1197.831547,-959.248284,1342.118844,0.998555,0.0,358.043829,2341.881156,0.0,186.275015,327.839976,2549.462252,102.743046,159.865021,660.372381,0.0,0.0,0.333855,0.0,0.666145,0.0,0.000000,0.0,0.0,2.546723,0.000153,1


In [ ]:
y_hat=test.copy()
y_hat.drop(['ID','date','burn_area','area'],axis=1,inplace=True)
y_hat.head()

,lat,lon,climate_aet,climate_def,climate_pdsi,climate_pet,climate_pr,climate_ro,climate_soil,climate_srad,climate_swe,climate_tmmn,climate_tmmx,climate_vap,climate_vpd,climate_vs,elevation,landcover_0,landcover_1,landcover_2,landcover_3,landcover_4,landcover_5,landcover_6,landcover_7,landcover_8,population_density,precipitation,month
0,25.447,5.296,142.756021,1217.926180,-953.401012,1360.571291,1.000000,0.0,355.593690,2349.499157,0.0,187.541787,332.578998,2542.124518,111.013969,156.563945,590.564102,0.0,0.0,0.430648,0.0,0.569045,0.0,0.000307,0.0,0.0,2.773423,0.000457,1
1,25.669,5.293,154.836847,1193.866315,-961.703043,1348.688788,1.000000,0.0,382.206636,2340.597389,0.0,185.895903,330.826066,2551.085649,106.576545,158.165429,611.544277,0.0,0.0,0.494624,0.0,0.505068,0.0,0.000307,0.0,0.0,4.697883,0.000129,1
2,25.443,5.074,127.790933,1223.640222,-962.435857,1351.521823,1.000000,0.0,318.078852,2351.867856,0.0,188.441645,330.077285,2551.183265,107.104292,153.257656,594.188987,0.0,0.0,0.288343,0.0,0.711657,0.0,0.000000,0.0,0.0,2.488614,0.000398,1
3,25.665,5.070,136.924571,1201.258784,-961.326298,1338.120278,1.000000,0.0,339.386497,2348.509414,0.0,185.393692,326.942319,2539.815326,101.962346,155.518048,645.362481,0.0,0.0,0.492103,0.0,0.507897,0.0,0.000000,0.0,0.0,2.439959,0.000158,1
4,25.886,5.067,144.278146,1197.831547,-959.248284,1342.118844,0.998555,0.0,358.043829,2341.881156,0.0,186.275015,327.839976,2549.462252,102.743046,159.865021,660.372381,0.0,0.0,0.333855,0.0,0.666145,0.0,0.000000,0.0,0.0,2.546723,0.000153,1


In [ ]:
newPred=model.predict(y_hat)

ValueError: ignored

In [ ]:
dicto={'ID':test.ID,'Prediction':newPred}

In [ ]:
df=pd.DataFrame(dicto)
df

In [ ]:
df.to_csv('submiss.csv',index=False)